In [2]:
import numpy as np 
import os
import tensorflow as tf
from tensorflow.keras.models import *
from tqdm import tqdm
from random import Random
from model import *

In [ ]:
model = get_model(8, 4)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = weighted_binary_crossentropy, metrics = [tf.keras.metrics.BinaryAccuracy(threshold=0.912), tf.keras.metrics.BinaryIoU(threshold=0.912)])
model.summary()

In [5]:
# path to preprocessed training data created from preprocessing_training_data.ipynb
PATH = 'data/train_processed/'

In [ ]:
# used to calculate weight_zero for the static_weighted_binary_crossentropy function
# value = fraction of one pixels / total pixels in training dataset

all_data = os.listdir(PATH)

# shuffling data before train/validation split
Random(333).shuffle(all_data)

SPLIT_train_val = int(len(all_data) * 0.7)
rolling_sum = 0
for num, img in tqdm(enumerate(all_data[:SPLIT_train_val])):
    temp = np.load(os.path.join(PATH, img))['mask'].astype(np.uint8)
    rolling_sum += temp.sum()

print(rolling_sum / (SPLIT_train_val * 484**2))

In [ ]:
# trains the model

BATCH_SIZE = 16

all_data = os.listdir(PATH)
all_data.sort()

# seeded shuffling data before train/validation split
Random(333).shuffle(all_data)

# 70% training, 15% validation, 15% testing
SPLIT_train_val = int(len(all_data) * 0.7)
SPLIT_val_test = int(len(all_data) * 0.85)

# iterable function that returns (img, mask) tuples from the training data. Performs augmentation on the data (normal, horizontal flip, vertical flip, horizontal & vertical flip)
def iter_train():
    for num in range(0, SPLIT_train_val * 4):
        index = int(num / 4)
        rem = num % 4
        npz = np.load(os.path.join(PATH, all_data[index]))
        img = npz['mdgm'].astype(np.uint8)
        mask = npz['mask'].astype(bool)
        if rem == 0:
            pass
        elif rem == 1:
            img = np.flip(img, axis=0)
            mask = np.flip(mask, axis=0)
        elif rem == 2:
            img = np.flip(img, axis=1)
            mask = np.flip(mask, axis=1)
        else:
            img = np.flip(np.flip(img, axis=0), axis=1)
            mask = np.flip(np.flip(mask, axis=0), axis=1)
        yield img, mask

def iter_validate():
    for num in range(SPLIT_train_val, SPLIT_val_test):
        npz = np.load(os.path.join(PATH, all_data[num]))
        img = npz['mdgm'].astype(np.uint8)
        mask = npz['mask'].astype(bool)
        yield img, mask

data_train = tf.data.Dataset.from_generator(iter_train, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

data_validate = tf.data.Dataset.from_generator(iter_validate, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# optional
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss"),
    tf.keras.callbacks.TensorBoard(
        log_dir="cloud_model", histogram_freq=1, write_images=True, update_freq=200
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "checkpoints/cloud_model_weights.ckpt", verbose=1, save_best_only=True
    ),
]

results = model.fit(
    data_train, validation_data=data_validate, verbose=1, epochs=250, callbacks=callbacks
)

# for tensorboard run command in activated python environment: python -m tensorboard.main --logdir="path\to\cloudmask\cloud_model" --host=127.0.0.1

In [ ]:
# saves the model
model.load_weights("checkpoints/cloud_model_weights.ckpt/")
model.save("save_cloud_model")

In [ ]:
# tests the model

all_data = os.listdir(PATH)
all_data.sort()

# seeded shuffling data before train/validation split
Random(333).shuffle(all_data)

# 70% training, 15% validation, 15% testing
SPLIT_val_test = int(len(all_data) * 0.85)

model = tf.keras.models.load_model('save_cloud_model', custom_objects={'weighted_binary_crossentropy': weighted_binary_crossentropy})

def iter_test():
    for num in range(SPLIT_val_test, len(all_data)):
        npz = np.load(os.path.join(PATH, all_data[num]))
        img = npz['mdgm'].astype(np.uint8)
        mask = npz['mask'].astype(bool)
        yield img, mask

BATCH_SIZE=1
data_test = tf.data.Dataset.from_generator(iter_test, output_signature=(
    tf.TensorSpec(shape=(668, 668, 3), dtype=tf.uint8),
    tf.TensorSpec(shape=(484, 484, 1), dtype=tf.bool)
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

res = model.evaluate(data_test)
print(res)